##Overview

This Notebook is a variation on the 2nd notebook in the project, implementing the candidates model.

The difference is in the creation of the user complex embedding.\
Instead of averaging over the embeddings of the viewing history movies, an RNN unit is applied to the history.
Several modifications had to be made to accomedate the RNN:
1. The user history is of constant size 50 with padding when necessary. This is kept the same, but the 0's padding is now to the left of the history. It was verfied that this has no effect over the nominal results, so all was handled correctly.
2. Dealing with 0 embeddings had to change. If before we could just not take the 0 embedding in the average, now we are forcing the 0 embedding to be a vector of 0s always, to not interfere with the RNN result.

### RNN variations and outputs
We tried both GRU and LSTM as RNN unit.\
The configuration was always one-directional with 2 layers.

For final RNN representation, we tried using both the last hidden state, as well as the average of all stages.

### Results

1. We were not able to improve the results beyond those of the basic averaging model.
2. LSTM performed much better than the GRU and got close to our best results with 40.3% HR@200



**Imports and administration**

In [1]:
# basic
from google.colab import runtime
import os 
import sys
import math
from time import time
import zipfile
import requests
import pickle
import gdown
!pip install --upgrade --no-cache-dir gdown

# general
import warnings
import numpy as np
import pandas as pd
# !pip install scikit-learn
# from sklearn.neighbors import LSHForest
# from sklearn import neighbors.LSHForest
# !apt install libomp-dev
# !python -m pip install --upgrade faiss faiss-gpu
# !pip install faiss-gpu
# import faiss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import average_precision_score

# visual
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

# notebook
from IPython.display import display, HTML
from tqdm import tqdm
import copy


# torch
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Sequential
from torch.nn import Sigmoid,ReLU
from torch.nn import Embedding,Linear,Dropout
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor,Compose
from torch.optim import SparseAdam,Adam,Adagrad,SGD

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
device

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


**Loading the datasets**

In [ ]:
# path = './drive/My Drive/Colab Notebooks/recsys_final_project/train_10_gru'
# with open(path , 'rb') as f:
#   (train, test, val, n_items, max_watches, num_neg_samples) = pickle.load(f)
#   print('loaded training')

loaded training


In [2]:
url = "https://drive.google.com/file/d/1-0YxUZUm3Vw3HzTCB4RHrH7upK9gcnXb/view?usp=sharing"
datasets_path = 'gru_datasets'
gdown.download(url, datasets_path, quiet=False,fuzzy=True)

# path = './drive/My Drive/Colab Notebooks/recsys_final_project/train_10_gru'
with open(datasets_path , 'rb') as f:
  (train, test, val, n_items, max_watches, num_neg_samples) = pickle.load(f)
  print('loaded training')

Downloading...
From: https://drive.google.com/uc?id=1-0YxUZUm3Vw3HzTCB4RHrH7upK9gcnXb
To: /content/gru_datasets
100%|██████████| 402M/402M [00:02<00:00, 193MB/s]


loaded training


In [ ]:
n_users = 6040

In [ ]:
class Candidates(torch.nn.Module):
    def __init__(self, config):
        super(Candidates, self).__init__()

        self.num_items = config['num_items']
        self.n_neg_samples = config['n_neg_samples']
        self.max_watches = config['max_watches']
        self.latent_dim = config['latent_dim']
        self.features_dim = config['features_dim']
        self.layers = config['layers']

        self.embed_items = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_dim) #log 4000 ~12
        # with torch.no_grad():
        #   self.embed_items.weight[0,:] = torch.zeros(self.latent_dim)#.to(device)
        self.embed_occ = torch.nn.Embedding(num_embeddings=21, embedding_dim=5) # 21 occupations log~5
        self.embed_age = torch.nn.Embedding(num_embeddings=7, embedding_dim=3) # 7 occupations log~3

        # self.rnn = nn.GRU(input_size = self.latent_dim, hidden_size = self.latent_dim, \
        #             num_layers = 2, batch_first = True, bidirectional=False) 
        
        self.rnn = nn.LSTM(input_size = self.latent_dim, hidden_size = self.latent_dim, \
                    num_layers = 2, batch_first = True, bidirectional=False) 

        self.fc_layers = torch.nn.ModuleList()
        self.fc_layers.append(torch.nn.Linear(self.features_dim, self.layers[0])) #linking features to MLP
        if len(self.layers) > 1:
          for (in_size, out_size) in zip(self.layers[:-1], self.layers[1:]):
              self.fc_layers.append(torch.nn.Linear(in_size, out_size))
        
        # self.LSHIndex = LSHIndex()
        self.N_candidates = config['N_candidates']
        self.knn = KNeighborsClassifier(n_neighbors=self.N_candidates) #initiating the KNN model with maximal N_candidates

    def forward(self, features, mode):
        # previous watches embedding and averaging
        # We must take into account the possible padding with 0's of the user history
        n = self.max_watches
        previous_watches = features[:, :n].int()
        with torch.no_grad():
          self.embed_items.weight[0,:] = torch.zeros(self.latent_dim)#.to(device)
        embedded_items = self.embed_items(previous_watches)
        embedded_items_rnn, ht = self.rnn(embedded_items)

        last_hidden_rnn = embedded_items_rnn[:,-1, :] # Taking the last hidden state

        # # optional averaging of RNN outputs
        # mask = torch.unsqueeze((previous_watches > 0),2)
        # sum = mask.sum(dim=1)
        # sum[sum == 0] = 1 #so we don't divide by 0 and get nan (even though overwritten)
        # embedded_items_mean = (embedded_items_rnn*mask).sum(dim=1)/sum
        # n_indices = torch.count_nonzero(previous_watches, dim=1)
        # embedded_items_mean[(n_indices == 0)] = torch.zeros(self.latent_dim).to(device)

        # other embeddings
        user_age = features[:, n].int()
        emb_age = self.embed_age(user_age)
        user_occupation = features[:, n+1].int()
        emb_occ = self.embed_occ(user_occupation)
        
        # other features
        other_features = features[:, n+2:n+5]
        
        # all
        # vector = torch.hstack((embedded_items_mean, emb_age, emb_occ, other_features)).to(torch.float32)
        vector = torch.hstack((last_hidden_rnn, emb_age, emb_occ, other_features)).to(torch.float32)

        for idx, _ in enumerate(range(len(self.fc_layers)-1)):
            vector = self.fc_layers[idx](vector)
            vector = torch.nn.ReLU()(vector)
        vector = self.fc_layers[-1](vector) #last layer without RELU

        if mode == 'training':
          item_id_label = features[:, -self.n_neg_samples-1].int()
          negative_samples = features[:, -self.n_neg_samples:].int() #negative samples are precalculated

          sample_indices = torch.hstack([torch.unsqueeze(item_id_label, 1), negative_samples])
          sample_embeddings = self.embed_items(sample_indices)
          dot_products = torch.matmul(sample_embeddings, torch.unsqueeze(vector, dim = 2))
          return dot_products.squeeze()
    
        elif mode == 'serving':
          # candidates = self.lshf.kneighbors(vector, n_neighbors=self.N_candidates, return_distance=False)
          # candidates = self.LSHIndex.query(vector, k = self.N_candidates)
          candidates = self.knn.kneighbors(vector.detach().cpu().numpy(), return_distance=False)
          return candidates #only indices

In [ ]:
class Training(object):

  def __init__(self, model, config):
    self.config = config
    self.model = model.to(self.config['device'])
    self.n_neg_samples = config['n_neg_samples']
    self.labels = torch.hstack((torch.tensor(1), torch.zeros(self.n_neg_samples))).repeat(config['batch_size'], 1).to(device)
    #importance weights with ratio of ~3000/100 = 30
    self.importance_weights = torch.hstack((torch.tensor(1), 30*torch.ones(self.n_neg_samples))).to(device)
    self.optimizer = config['optimizer_type'](model.parameters(), **config['optimizer_parameter'])
    self.criterion = config['criterion'](weight = self.importance_weights) #here we add importance weights to the loss function
    self.dl_train = DataLoader(train, batch_size=config['batch_size'], shuffle=True) # create dataloader with given batch size
    self.dl_val = DataLoader(val, batch_size=config['batch_size'], shuffle=False) # create dataloader with given batch size
    self.knn_labels = np.ones(self.model.num_items)
    self.AP_labels = torch.tensor([1, 0]).repeat(1, n_users).squeeze()

  def train(self):
    self.train_loss_history = []
    self.eval_loss_history = []
    self.eval_map_history = []
    self.eval_HR100_history = []
    self.eval_HR200_history = []
    self.eval_MRR100_history = []
    self.eval_MRR200_history = []
    self.eval_NDCG100_history = []
    self.eval_NDCG200_history = []

    epochs_without_improvement = 0
    best_HR = None 
    train_start = time()
    for epoch in range(self.config['n_epochs']):
      self.train_epoch() #train
      self.train_loss_history.append(self.epoch_train_loss/len(self.dl_train))
      # extract all embeddings
      all_embeddings = self.model.embed_items(torch.arange(0, n_items).to(device)).detach().cpu().numpy()
      self.model.knn.fit(all_embeddings, self.knn_labels)
      # self.model.LSHIndex.build(all_embeddings)
      # self.model.lshf.kneighbors.fit(all_embeddings)
      if epoch%1 == 0:
      # if epoch%10 == 0:
        self.evaluate_epoch(self.dl_val) #evaluate
        # aggregate metrics: note len(val) = len(test)
        self.eval_loss_history.append(self.epoch_eval_loss/len(self.dl_val))
        average_precision = average_precision_score(self.AP_labels, self.epoch_pred)
        self.eval_map_history.append(average_precision)
        self.eval_HR100_history.append(self.epoch_HR100/len(val))
        self.eval_HR200_history.append(self.epoch_HR200/len(val))
        self.eval_MRR100_history.append(self.epoch_MRR100/len(val))
        self.eval_MRR200_history.append(self.epoch_MRR200/len(val))
        self.eval_NDCG100_history.append(self.epoch_NDCG100/len(val))
        self.eval_NDCG200_history.append(self.epoch_NDCG200/len(val))
        print(f'epoch {epoch}: loss = {self.train_loss_history[-1]}, HR@200 = {self.eval_HR200_history[-1]}, AP = {self.eval_map_history[-1]}')
        # print(f'epoch {epoch}: loss = {self.train_loss_history[-1]}, HR@200 = {self.eval_HR200_history[-1]}')
        #check for early stopping
        if not best_HR or self.eval_HR200_history[-1] > best_HR:
          best_HR = self.eval_HR200_history[-1]
          # best_MRR200 = self.eval_MRR200_history[-1]
          # best_NDCG200 = self.eval_NDCG200_history[-1]
          # best_loss = self.eval_loss_history[-1]
          epochs_without_improvement = 0
          #print ("Achieved lower validation loss, save model at epoch number {} ".format(epoch + 1) )
          best_model = copy.deepcopy(self.model.state_dict())
        else:
          epochs_without_improvement += 1

        if epochs_without_improvement == self.config['early_stopping']:
          if self.config['verbose']:
              print('\nEarly stoping after {} epochs. validation loss did not imporve for more than {} epcochs'.format(epoch, self.config['early_stopping']))
          break
    self.training_time = time() - train_start

    # load best model and best performance
    self.model.load_state_dict(best_model)
    if self.config['verbose']:
        print('\nFinished Training:')
        print('Best metrics are:')
        print(f'Hit Ratio eval = {best_HR}')
    
  def train_epoch(self):
    self.epoch_train_loss   = 0
    self.model.train() # train mode
    for batch in tqdm(self.dl_train, disable=(not self.config['verbose'])):
      self.train_batch(batch)

  def train_batch(self, batch):
     
    batch = batch.to(device)  
    pred = self.model(batch, mode = 'training')
    labels = self.labels
    if pred.shape[0] < self.config['batch_size']:
      labels = torch.hstack((torch.tensor(1), torch.zeros(self.n_neg_samples))).repeat(pred.shape[0], 1).to(device)
    loss = self.criterion(pred, labels)
              
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()               
    self.epoch_train_loss += loss.item()

  def evaluate_epoch(self, dl_eval):
    self.epoch_eval_loss = 0
    self.epoch_pred = torch.empty(0) #Aggregated for AP calculation
    self.epoch_HR100 = 0
    self.epoch_HR200 = 0
    self.epoch_MRR100 = 0
    self.epoch_MRR200 = 0
    self.epoch_NDCG100 = 0
    self.epoch_NDCG200 = 0

    self.model.eval() #evaluation mode
    with torch.no_grad():
      for batch in tqdm(dl_eval, disable=(not self.config['verbose'])):
        self.eval_batch(batch) # dl_val was built so that one batch is one user


  def eval_batch(self, batch):
    target_items = batch[:, -num_neg_samples-1]
    # Send tensor to GPU    
    batch = batch.to(device)

    #evaluations of candidates retrieval
    candidates_batch = self.model(batch, mode = 'serving') #already returned sorted from high to low
    for candidates, target_item in zip(candidates_batch, target_items):
      self.epoch_HR100 += self.HitRatio(candidates, target_item, 100)
      self.epoch_HR200 += self.HitRatio(candidates, target_item, 200)
      self.epoch_MRR100 += self.MRR(candidates, target_item, 100)
      self.epoch_MRR200 += self.MRR(candidates, target_item, 200)
      self.epoch_NDCG100 += self.NDCG(candidates, target_item, 100)
      self.epoch_NDCG200 += self.NDCG(candidates, target_item, 200)
  
    #AP evaluation using negative samples
    pred = self.model(batch, mode = 'training') # not really training - extracts negative samples prediction for AP calculation
    labels = self.labels
    if pred.shape[0] < self.config['batch_size']:
      labels = torch.hstack((torch.tensor(1), torch.zeros(self.n_neg_samples))).repeat(pred.shape[0], 1).to(device)
    loss = self.criterion(pred, labels)        
    self.epoch_eval_loss += loss.item()

    probabilities = F.softmax(pred.detach().cpu(), dim = 1)
    for prob in probabilities: #concatenating predictions for average precision calculation at the end of the epoch
      self.epoch_pred = torch.hstack((self.epoch_pred, prob[:2])) #first two here (positive and negative)


  def extract_candidates(self, dl_eval):
    self.model.eval() #evaluation mode
    all_candidates = np.empty((0, self.model.N_candidates))
    with torch.no_grad():
      for batch in tqdm(dl_eval, disable=(not self.config['verbose'])):
        target_items = batch[:, -num_neg_samples-1]
        # Send tensor to GPU    
        batch = batch.to(device)
        candidates_batch = self.model(batch, mode = 'serving')
        all_candidates = np.vstack([all_candidates, candidates_batch])
        # previous_watches = batch[:, :max_watches].int()
        # for candidates, prev_watches in zip(candidates_batch, previous_watches):
        #   #remove previous watches from candidates, since there are no duplicities in this dataset
        #   candidates = [x for x in candidates if x not in prev_watches]
      return all_candidates

  def HitRatio(self, ranked_items, target_item, k):
    for item in ranked_items[:k]:
      if item == target_item:
        return 1
    return 0

  def MRR(self, ranked_items, target_item, k):
    for i, item in enumerate(ranked_items[:k]):
      if item == target_item:
        return 1/(i + 1)
    return 0

  def NDCG(self, ranked_items, target_item, k):
    for i, item in enumerate(ranked_items[:k]):
      if item == target_item:
        return np.log(2)/np.log(i + 2)
    return 0


###Experiments

Below you can find learning processes for different configurations (in comment).\
The Best configuration achieved was with a 2 layers LSTM and MLP of layers = [1024, 512, 256].\
The test results for HR@200 were 40.3%, which is a little below the averaging model.\
We only run 61 epochs for all cases, which don't converge completely, but the behavior is still consistently lower than the nominal.

In [ ]:
# best_results = pd.DataFrame(columns=['maximal width', 'Batch Size', 'Learning Rate', 'Topk', 'Metric', 'Score']) #uncomment to run

In [ ]:
def add_results(training_model, results_df, max_width, batch_size, lr):

  MAP = average_precision_score(training_model.AP_labels, training_model.epoch_pred)
  loss = training_model.epoch_eval_loss/len(val)
  hr100 = training_model.epoch_HR100/len(val)
  hr200 = training_model.epoch_HR200/len(val)
  mrr100 = training_model.epoch_MRR100/len(val)
  mrr200 = training_model.epoch_MRR200/len(val)
  ndcg100 = training_model.epoch_NDCG100/len(val)
  ndcg200 = training_model.epoch_NDCG200/len(val)
  tr_time = training_model.training_time

  results_df.loc[len(results_df)] = max_width, batch_size, lr, 0, 'LOSS', loss
  results_df.loc[len(results_df)] = max_width, batch_size, lr, 0, 'MAP', MAP
  results_df.loc[len(results_df)] = max_width, batch_size, lr, 0, 'TIME', tr_time
  results_df.loc[len(results_df)] = max_width, batch_size, lr, 100, 'HR' , hr100
  results_df.loc[len(results_df)] = max_width, batch_size, lr, 200, 'HR', hr200
  results_df.loc[len(results_df)] = max_width, batch_size, lr, 100, 'MRR', mrr100
  results_df.loc[len(results_df)] = max_width, batch_size, lr, 200, 'MRR', mrr200
  results_df.loc[len(results_df)] = max_width, batch_size, lr, 100, 'NDCG', ndcg100
  results_df.loc[len(results_df)] = max_width, batch_size, lr, 200, 'NDCG', ndcg200

In [ ]:
batch_size = 50 # with 2 layers LSTM and 0 for 0 embedding - reverse padding
dl_test = DataLoader(test, batch_size=batch_size, shuffle=False)
lr = 0.001
layers = [1024, 512, 256] 

torch.manual_seed(42)
np.random.seed(42) 
maximal_width = layers[0]
Candidates_config = {'num_items': n_items, 'n_neg_samples': num_neg_samples, 'max_watches': max_watches, 'latent_dim': 256, 'features_dim': 267, 'layers':layers, 'N_candidates':250}
model_Candidates = Candidates(Candidates_config)

training_config = {'n_neg_samples': num_neg_samples, 'batch_size': batch_size, 'optimizer_type': Adam, 'optimizer_parameter': {'lr': lr}, \
              'criterion' : torch.nn.CrossEntropyLoss, 'n_epochs' : 61, 'early_stopping' : 8, 'verbose' : True, 'device' : device}


training_candidates = Training(model_Candidates, training_config)
training_candidates.train()

100%|██████████| 121/121 [00:55<00:00,  2.18it/s]


epoch 0: loss = 2.846972787735478, HR@200 = 0.13990066225165562, AP = 0.9178394396176548


100%|██████████| 121/121 [00:54<00:00,  2.22it/s]


epoch 1: loss = 2.1036979544439065, HR@200 = 0.1607615894039735, AP = 0.9350982177866177


100%|██████████| 121/121 [00:54<00:00,  2.23it/s]


...

epoch 58: loss = 0.5977184067326073, HR@200 = 0.4076158940397351, AP = 0.917754591969393


100%|██████████| 121/121 [00:46<00:00,  2.58it/s]


epoch 59: loss = 0.5938634243713605, HR@200 = 0.4067880794701987, AP = 0.9168994776312034


100%|██████████| 121/121 [00:47<00:00,  2.53it/s]

epoch 60: loss = 0.5910417292045047, HR@200 = 0.4100993377483444, AP = 0.9177285071465753

Finished Training:
Best metrics are:
Hit Ratio eval = 0.4100993377483444


In [ ]:
results = pd.DataFrame(columns=['maximal width', 'Batch Size', 'Learning Rate', 'Topk', 'Metric', 'Score']) #uncomment to run

In [ ]:
training_candidates.evaluate_epoch(dl_test)

100%|██████████| 121/121 [00:48<00:00,  2.49it/s]


In [ ]:
add_results(training_candidates, results, maximal_width, batch_size, lr)

In [ ]:
results

,maximal width,Batch Size,Learning Rate,Topk,Metric,Score
0,1024,50,0.001,0,LOSS,0.102132
1,1024,50,0.001,0,MAP,0.907252
2,1024,50,0.001,0,TIME,7128.634667
3,1024,50,0.001,100,HR,0.265894
4,1024,50,0.001,200,HR,0.403146
5,1024,50,0.001,100,MRR,0.014933
6,1024,50,0.001,200,MRR,0.015907
7,1024,50,0.001,100,NDCG,0.058599
8,1024,50,0.001,200,NDCG,0.077752


In [ ]:
path = './drive/My Drive/Colab Notebooks/recsys_final_candidates_LSTM'
with open(path, 'wb') as f:
  pickle.dump((results), f)

In [ ]:
batch_size = 50 # with 2 layers LSTM and 0 for 0 embedding - reverse padding and averaging lstm output
dl_test = DataLoader(test, batch_size=batch_size, shuffle=False)
lr = 0.001
layers = [1024, 512, 256] 

torch.manual_seed(42)
np.random.seed(42) 
maximal_width = layers[0]
Candidates_config = {'num_items': n_items, 'n_neg_samples': num_neg_samples, 'max_watches': max_watches, 'latent_dim': 256, 'features_dim': 267, 'layers':layers, 'N_candidates':250}
model_Candidates = Candidates(Candidates_config)

training_config = {'n_neg_samples': num_neg_samples, 'batch_size': batch_size, 'optimizer_type': Adam, 'optimizer_parameter': {'lr': lr}, \
              'criterion' : torch.nn.CrossEntropyLoss, 'n_epochs' : 61, 'early_stopping' : 8, 'verbose' : True, 'device' : device}


training_candidates = Training(model_Candidates, training_config)
training_candidates.train()

100%|██████████| 121/121 [00:55<00:00,  2.19it/s]


epoch 0: loss = 3.063849717141777, HR@200 = 0.1316225165562914, AP = 0.8986300806349167


100%|██████████| 121/121 [00:54<00:00,  2.21it/s]


epoch 1: loss = 2.2974756523276008, HR@200 = 0.1597682119205298, AP = 0.9168927263545821


100%|██████████| 121/121 [00:54<00:00,  2.24it/s]


...

epoch 10: loss = 1.2677859916789642, HR@200 = 0.28211920529801326, AP = 0.9337146340368321


100%|██████████| 121/121 [00:51<00:00,  2.36it/s]


epoch 11: loss = 1.230819908524586, HR@200 = 0.2955298013245033, AP = 0.9325130135120723


 14%|█▍        | 873/6040 [00:10<00:59, 86.25it/s]


KeyboardInterrupt: ignored

In [ ]:
batch_size = 50 # with gru and 0 for 0 embedding - reverse padding
dl_test = DataLoader(test, batch_size=batch_size, shuffle=False)
lr = 0.001
layers = [1024, 512, 256] 

torch.manual_seed(42)
np.random.seed(42) 
maximal_width = layers[0]
Candidates_config = {'num_items': n_items, 'n_neg_samples': num_neg_samples, 'max_watches': max_watches, 'latent_dim': 256, 'features_dim': 267, 'layers':layers, 'N_candidates':250}
model_Candidates = Candidates(Candidates_config)

training_config = {'n_neg_samples': num_neg_samples, 'batch_size': batch_size, 'optimizer_type': Adam, 'optimizer_parameter': {'lr': lr}, \
              'criterion' : torch.nn.CrossEntropyLoss, 'n_epochs' : 61, 'early_stopping' : 8, 'verbose' : True, 'device' : device}


training_candidates = Training(model_Candidates, training_config)
training_candidates.train()
# training_candidates.evaluate_epoch(dl_test)
# add_results(training_candidates, best_results, maximal_width, batch_size, lr)

100%|██████████| 121/121 [00:54<00:00,  2.23it/s]


epoch 0: loss = 2.7405855264490016, HR@200 = 0.14288079470198675, AP = 0.9204618166815888


100%|██████████| 121/121 [00:54<00:00,  2.22it/s]


epoch 1: loss = 2.067728934303814, HR@200 = 0.16655629139072847, AP = 0.9321732206347175


100%|██████████| 121/121 [00:54<00:00,  2.23it/s]


...

epoch 13: loss = 1.6575398274506166, HR@200 = 0.24420529801324503, AP = 0.9392743076699758


100%|██████████| 121/121 [00:51<00:00,  2.36it/s]


epoch 14: loss = 1.6753103645826808, HR@200 = 0.24254966887417218, AP = 0.9394640612386311


 22%|██▏       | 1335/6040 [00:14<00:51, 90.52it/s]


KeyboardInterrupt: ignored

In [ ]:
batch_size = 50
dl_test = DataLoader(test, batch_size=batch_size, shuffle=False)
lr = 0.001
layers = [1024, 512, 256] 

torch.manual_seed(42)
np.random.seed(42) 
maximal_width = layers[0]
Candidates_config = {'num_items': n_items, 'n_neg_samples': num_neg_samples, 'max_watches': max_watches, 'latent_dim': 256, 'features_dim': 267, 'layers':layers, 'N_candidates':250}
model_Candidates = Candidates(Candidates_config)

training_config = {'n_neg_samples': num_neg_samples, 'batch_size': batch_size, 'optimizer_type': Adam, 'optimizer_parameter': {'lr': lr}, \
              'criterion' : torch.nn.CrossEntropyLoss, 'n_epochs' : 61, 'early_stopping' : 8, 'verbose' : True, 'device' : device}


training_candidates = Training(model_Candidates, training_config)
training_candidates.train()

100%|██████████| 121/121 [00:57<00:00,  2.09it/s]


epoch 0: loss = 2.9936776732372135, HR@200 = 0.12019867549668874, AP = 0.8903327626110267


100%|██████████| 121/121 [00:55<00:00,  2.19it/s]


epoch 1: loss = 2.3623286915733326, HR@200 = 0.148841059602649, AP = 0.9012660051509083


100%|██████████| 121/121 [00:53<00:00,  2.24it/s]


...

epoch 12: loss = 1.2263056888485586, HR@200 = 0.298841059602649, AP = 0.8913500722439394


100%|██████████| 121/121 [00:50<00:00,  2.41it/s]


epoch 13: loss = 1.1844409492434256, HR@200 = 0.3109271523178808, AP = 0.8905889548341444


 36%|███▋      | 44/121 [00:17<00:31,  2.47it/s]


KeyboardInterrupt: ignored

In [ ]:
batch_size = 50 # with gru and no constraint on 0 embedding - reverse padding
dl_test = DataLoader(test, batch_size=batch_size, shuffle=False)
lr = 0.001
layers = [512, 256] 

torch.manual_seed(42)
np.random.seed(42) 
maximal_width = layers[0]
Candidates_config = {'num_items': n_items, 'n_neg_samples': num_neg_samples, 'max_watches': max_watches, 'latent_dim': 256, 'features_dim': 267, 'layers':layers, 'N_candidates':250}
model_Candidates = Candidates(Candidates_config)

training_config = {'n_neg_samples': num_neg_samples, 'batch_size': batch_size, 'optimizer_type': Adam, 'optimizer_parameter': {'lr': lr}, \
              'criterion' : torch.nn.CrossEntropyLoss, 'n_epochs' : 61, 'early_stopping' : 8, 'verbose' : True, 'device' : device}


training_candidates = Training(model_Candidates, training_config)
training_candidates.train()

100%|██████████| 121/121 [00:53<00:00,  2.27it/s]


epoch 0: loss = 2.735156523293217, HR@200 = 0.15529801324503312, AP = 0.9194635095057576


100%|██████████| 121/121 [00:50<00:00,  2.39it/s]


epoch 1: loss = 2.077304863475806, HR@200 = 0.1759933774834437, AP = 0.9304382272676134


100%|██████████| 121/121 [00:51<00:00,  2.35it/s]


...

epoch 14: loss = 1.6904558524193354, HR@200 = 0.2683774834437086, AP = 0.9361114313682839


100%|██████████| 121/121 [00:48<00:00,  2.48it/s]


epoch 15: loss = 1.7136366772631935, HR@200 = 0.26589403973509934, AP = 0.9368789186898461


 15%|█▍        | 18/121 [00:06<00:38,  2.65it/s]


KeyboardInterrupt: ignored

In [ ]:
batch_size = 50 # with gru and no constraint on 0 embedding
dl_test = DataLoader(test, batch_size=batch_size, shuffle=False)
lr = 0.001
layers = [512, 256] 

torch.manual_seed(42)
np.random.seed(42) 
maximal_width = layers[0]
Candidates_config = {'num_items': n_items, 'n_neg_samples': num_neg_samples, 'max_watches': max_watches, 'latent_dim': 256, 'features_dim': 267, 'layers':layers, 'N_candidates':250}
model_Candidates = Candidates(Candidates_config)

training_config = {'n_neg_samples': num_neg_samples, 'batch_size': batch_size, 'optimizer_type': Adam, 'optimizer_parameter': {'lr': lr}, \
              'criterion' : torch.nn.CrossEntropyLoss, 'n_epochs' : 61, 'early_stopping' : 8, 'verbose' : True, 'device' : device}


training_candidates = Training(model_Candidates, training_config)
training_candidates.train()
# training_candidates.evaluate_epoch(dl_test)
# add_results(training_candidates, best_results, maximal_width, batch_size, lr)

100%|██████████| 121/121 [00:53<00:00,  2.27it/s]


epoch 0: loss = 3.0310574258400114, HR@200 = 0.13956953642384107, AP = 0.9079621037539802


100%|██████████| 121/121 [00:51<00:00,  2.33it/s]


epoch 1: loss = 2.347568300682188, HR@200 = 0.15529801324503312, AP = 0.9213751951612015


100%|██████████| 121/121 [00:52<00:00,  2.32it/s]


...

epoch 12: loss = 1.849354935600268, HR@200 = 0.2456953642384106, AP = 0.9266780946599538


100%|██████████| 121/121 [00:50<00:00,  2.41it/s]


epoch 13: loss = 1.8632454869960318, HR@200 = 0.24867549668874173, AP = 0.9262024919286147


 29%|██▉       | 1767/6040 [00:18<00:44, 96.05it/s] 


KeyboardInterrupt: ignored

In [ ]:
batch_size = 50 # with gru and no constraint on 0 embedding
dl_test = DataLoader(test, batch_size=batch_size, shuffle=False)
lr = 0.001
layers = [1024, 512, 256] 

torch.manual_seed(42)
np.random.seed(42) 
maximal_width = layers[0]
Candidates_config = {'num_items': n_items, 'n_neg_samples': num_neg_samples, 'max_watches': max_watches, 'latent_dim': 256, 'features_dim': 267, 'layers':layers, 'N_candidates':250}
model_Candidates = Candidates(Candidates_config)

training_config = {'n_neg_samples': num_neg_samples, 'batch_size': batch_size, 'optimizer_type': Adam, 'optimizer_parameter': {'lr': lr}, \
              'criterion' : torch.nn.CrossEntropyLoss, 'n_epochs' : 61, 'early_stopping' : 8, 'verbose' : True, 'device' : device}


training_candidates = Training(model_Candidates, training_config)
training_candidates.train()
# training_candidates.evaluate_epoch(dl_test)
# add_results(training_candidates, best_results, maximal_width, batch_size, lr)

100%|██████████| 121/121 [00:52<00:00,  2.30it/s]


epoch 0: loss = 2.8968589526730657, HR@200 = 0.1369205298013245, AP = 0.9157254486963474


100%|██████████| 121/121 [00:52<00:00,  2.30it/s]


epoch 1: loss = 2.166946370613496, HR@200 = 0.16605960264900663, AP = 0.9299226641349213


100%|██████████| 121/121 [00:50<00:00,  2.38it/s]


...

epoch 14: loss = 1.7268253048030746, HR@200 = 0.22582781456953643, AP = 0.9400903669612881


100%|██████████| 121/121 [00:51<00:00,  2.37it/s]


epoch 15: loss = 1.749784088332132, HR@200 = 0.22201986754966888, AP = 0.9384861767786492


 75%|███████▍  | 4503/6040 [00:48<00:16, 92.04it/s]


KeyboardInterrupt: ignored